In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "drive/MyDrive/Colab Notebooks/"
!kaggle datasets download -d jessicali9530/celeba-dataset

from google.colab import drive
drive.mount('content/drive')


import os
import numpy as np

Paths = os.listdir('img_align_celeba/img_align_celeba')

images = []

for path in Paths[0:50000]:
  digitization = PIL.Image.open('img_align_celeba/img_align_celeba/' + path).crop((20,30,160,180)).convert('L').resize((64,64))
  images.append( np.array(digitization) )

images = np.array(images)
images = np.divide(images, 255)

images = images.reshape( 50000,64,64,1 )
print(images.shape)

In [ ]:
import tensorflow as tf

discriminator = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=[64,64,1]),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

generator = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4 * 4 * 256, input_shape=(100,) ),
  tf.keras.layers.Reshape((4, 4, 256)),
  tf.keras.layers.Conv2DTranspose(256, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(128, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same'),
  tf.keras.layers.LeakyReLU(alpha=0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation='sigmoid')
])

GAN = tf.keras.models.Sequential([generator, discriminator])

discriminator.compile(
    optimizer='adam',
    loss='binary_crossentropy'
)
discriminator.trainable = False

GAN.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
x_data = images

for i in range(50000//128):

  #discriminator
  Random_number = np.random.uniform(-1,1,size=(128,100))
  fake = generator.predict(Random_number)
  real = x_data[0:128]

  fakeY = np.ones(shape=(128,1))
  realY = np.zeroes(shape=(128,1))

  loss1 = discriminator.train_on_batch(fake, fakeY)
  loss2 = discriminator.train_on_batch(real, 진realY짜사진Y)


  #generator
  noise = np.random.uniform(-1,1,size=(128,100))
  Y = np.ones(shape=(128,1))
  loss3 = GAN.train_on_batch(noise, Y)


  Random_number = np.random.uniform(-1,1,size=(10,100))
predict = generator.predict(Random_number)

for i in range(10):
  plt.subplot(2, 5, i+1)
  plt.imshow(predict[i].reshape(64,64), cmap='gray')
  plt.axis('off')
plt.tight_layout()
plt.show()